<a href="https://colab.research.google.com/github/swilsonmfc/pandas/blob/main/8_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Bookings

![](https://cdn.inprnt.com/thumbs/5b/fa/5bfa6a8ea6c461d1c21c0a44962968ed.jpg?response-cache-control=max-age=2628000)

# Setup

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.graphics.regressionplots as regplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data
* https://www.kaggle.com/jessemostipak/hotel-booking-demand

In [32]:
!wget -O hotel.zip https://github.com/swilsonmfc/pandas/blob/main/hotel.zip?raw=true

--2021-07-23 18:11:21--  https://github.com/swilsonmfc/pandas/blob/main/hotel.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/swilsonmfc/pandas/raw/main/hotel.zip [following]
--2021-07-23 18:11:21--  https://github.com/swilsonmfc/pandas/raw/main/hotel.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/swilsonmfc/pandas/main/hotel.zip [following]
--2021-07-23 18:11:21--  https://raw.githubusercontent.com/swilsonmfc/pandas/main/hotel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1308365 (1.2M) [applic

In [33]:
!unzip -o hotel.zip

Archive:  hotel.zip
replace hotel_bookings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [34]:
hotel_df = pd.read_csv('./hotel_bookings.csv')

In [35]:
hotel_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


## Types

In [36]:
hotel_df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

## Missing
* Missing data here seems reasonable for the most part
* 4 missing children - Impute to median

In [37]:
hotel_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [38]:
# Series where method vs np.where
hotel_df['children'] = hotel_df.children.where(hotel_df.children.notna(), 0)
hotel_df['country']  = hotel_df.country.fillna('UNKNOWN')
hotel_df['agent']    = hotel_df.agent.fillna('NONE')
hotel_df['company']  = hotel_df.company.fillna('NONE')

In [39]:
hotel_df.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

## Outliers
* More than 10 people booked in a room
  * All cancelled
  * Agent 96 is present on 1/2 of them
* The hotel paid one person to stay there
  * Best to truncate that to $0
  * One room cost \$5,400

In [40]:
hotel_df = hotel_df[(hotel_df.adr >= 0) & (hotel_df.adr < 1000)]

# Logistic Regression

In [41]:
f = 'is_canceled ~ hotel + arrival_date_month + deposit_type'
model = smf.logit(formula=f, data=hotel_df)
fitted = model.fit()
print(fitted.summary())

Optimization terminated successfully.
         Current function value: 0.523298
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:            is_canceled   No. Observations:               119388
Model:                          Logit   Df Residuals:                   119373
Method:                           MLE   Df Model:                           14
Date:                Fri, 23 Jul 2021   Pseudo R-squ.:                  0.2061
Time:                        18:11:36   Log-Likelihood:                -62475.
converged:                       True   LL-Null:                       -78698.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          -0.7494      0.023    -32.483  

In [42]:
f = """is_canceled ~ + C(arrival_date_month, Treatment(reference='January'))
             + C(hotel, Treatment(reference='Resort Hotel')) 
             + C(deposit_type)""" 
model = smf.logit(formula=f, data=hotel_df)
fitted = model.fit()
print(fitted.summary())

Optimization terminated successfully.
         Current function value: 0.523298
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:            is_canceled   No. Observations:               119388
Model:                          Logit   Df Residuals:                   119373
Method:                           MLE   Df Model:                           14
Date:                Fri, 23 Jul 2021   Pseudo R-squ.:                  0.2061
Time:                        18:11:39   Log-Likelihood:                -62475.
converged:                       True   LL-Null:                       -78698.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                                         coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------

In [43]:
# Generate a single prediction
pred_df = pd.DataFrame([{'hotel': 'City Hotel', 'arrival_date_month': 'July', 'deposit_type': 'Refundable'}])
proba = fitted.predict(pred_df)
pred  = np.where(proba < .5, 0, 1) # Round works here too!
pred

array([0])

In [44]:
# Generate a prediction for a resort hotel in October with non-refundable ticket (Non Refund)
pred_df = pd.DataFrame([{'hotel': 'Resort Hotel', 'arrival_date_month': 'October', 'deposit_type': 'Non Refund'}])


In [45]:
# Predict over the entire dataset

In [46]:
# Produce a confusion matrix

In [47]:
# Print the classification report

# Sklearn Logistic Regression

In [48]:
# Generate a dataframe of is_canceled, lead_time, adr & deposit type
# Target variable is is_canceled

In [49]:
# Filter the dataset to Portugal (country == PRT)

In [50]:
# Preprocess deposit type as one hot encoded

In [51]:
# Train - test split 

In [52]:
# Scale the lead time & adr

In [53]:
# Compute class weights for balancing

In [54]:
# Construct a Linear Regression Object & Fit to the training data
logit = LogisticRegression()
# logit.fit(...)

In [55]:
# Predict on the test dataset

In [56]:
# Print the classification report

# Sklearn Linear Regression

In [57]:
# Generate a dataframe of adr, lead_time, and country
# Target variable is adr

In [58]:
# Preprocess country as one hot encoded
# Note:  add the parameter drop_first=True to get_dummies

In [59]:
# Train - test split 

In [61]:
# Construct a Linear Regression Object & Fit to the training data
lin = LinearRegression()
# lin.fit(...)

In [62]:
# Predict on the test dataset

In [63]:
# Compute the mean absolute error 